INPUTS
- `sset.PATH_CIAM_COASTLINES_BY_ISO`
- `sset.PATH_CIAM_SITES_VORONOI_BY_ISO`

OUTPUTS
- `sset.PATH_CIAM_COASTLINES_BY_CIAM_SITE`

In [ ]:
from sliiders import settings as sset
from sliiders import spatial

spatial.filter_spatial_warnings()

In [ ]:
coastal_segs = spatial.get_coastal_segments_by_ciam_site(
    sset.PATH_CIAM_SITES_VORONOI_BY_ISO,
    sset.PATH_CIAM_COASTLINES_BY_ISO
)

### Save

In [ ]:
coastal_segs.to_parquet(sset.PATH_CIAM_COASTLINES_BY_CIAM_SITE, index=False)